#### 

Goal: Review upcoming weather to determine when would be the best time period for someone to run.   
- Agent 1: Extract longitude and latitude coordinates from city in prompt, return weather 
- Agent 2: Evaluate when is best time period to run

### Import Credentials

In [ ]:
# !pip install openai
# !pip install requests

In [1]:
from creds import credentials
import calendars
import weather
import json
from openai import OpenAI
import os

/Users/jparks/Repositories/tool-calling-OpenAI/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# get keys
openai_key = credentials['tool-calling-OpenAI']
weather_api = credentials['weather_api']

## *Agent #1: Extract location and get weather*

In [4]:
# 1A: Define function

weather_tool = [{
    # weather tool
        "type": "function",
        "name": "get_weather",
        "description":"get temperature, humidity, precipitation, wind speed for specified location for the next 5 days.",
        "parameters":{
            # param names + types
            "type": "object",
            "properties": {
                "lat": {"type":"number"},
                "long":{"type":"number"}
            },
            # all params in required b/c strict = True
            "required": ["lat", "long"],
            "additionalProperties": False
        },
    # recommended to always use true
    # does not allow model to skip parameteres
    "strict" : True
}]

### *Agent #2: Get Upcoming calendar events*

In [5]:
calendar_get_events = [{
    # weather tool
        "type": "function",
        "name": "get_upcoming_event",
        "description":"Get the days and times from teh user calendar of when they will be busy in the upcoming days. The furthest you can look ahead is 5 days.",
        "parameters":{
            # param names + types
            "type": "object",
            "properties": {
                "cal_id": {"type":"string",
                           "description":"""ID of the calendar to use. It is a random combination of letters and numbers. 
                           Do not use this parameter if they do not give an ID."""},
                "in_future":{"type":"number",
                             "description":"Number of days to look ahead. Do not fill in if the user does not provide it, there is a default value in function."}
            },
            # all params in required when strict = True
            # "required": ["cal_id", "in_future"],
            "additionalProperties": False
        },
    # setting to False
    # so blank string doen't override defaults
    "strict" : False
}]

### *Agent 3: Write Event to Schedule*

In [6]:
schedule_event = [{
    # create event
        "type": "function",
        "name": "write_event",
        "description":"""Schedule the event passed to you.
                        Please convert the length of the event into hours.""",
        "parameters":{
            # param names + types
            "type": "object",
            "properties": {
                "start_time": {"type":"string",
                           "description":"""When the event should begin.
                                            The output of the time should follow the format of '%Y-%m-%dT%H:%M:%S'."""},
                "length_of_meeting":{"type":"number",
                             "description":"Number of days to look ahead. Do not fill in if the user does not provide it, there is a default value in function."}
            },
            # all params in required when strict = True
            "required": ["start_time", "length_of_meeting"],
            "additionalProperties": False
        },
    # setting to False
    # so blank string doen't override defaults
    "strict" : True
}]

### *Agent #4: Review the weather and evaluate when it would be best to run outside*

Step 1: Get the input message

In [ ]:
# Message
input_message = [{
    "role":"user",
    "content": """I live in Anchorage, Alaska. "
    "What is the best day and time to go for a 30 minute run based on the weather and my work schedule for the next 2 days? "
    "Please schedule it between meetings and during early afternoon, before 5 p.m.""",
}]

Step 2: Input the message along with the tools that can be used.

In [ ]:
# create client
client = OpenAI(api_key=openai_key)

response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_message,
    tools=(calendar_get_events + weather_tool)
)

Step 3: Create a function that will extract the parameters that Step 2 outputs, so we can plug the parameters into the tool call functions.

In [ ]:
# function to take argument outputs from model
# and plug them into the function to get their return
def call_function(name, args):
    if name == "get_weather":
        return weather.get_weather(**args)
    if name == "get_upcoming_event":
        return calendars.get_upcoming_events(**args)

Step 4: Plug parameters into tools, save outputs.

In [ ]:
for tool_call in response.output:
    if tool_call.type != "function_call":
        continue
    
    # get function name
    name = tool_call.name
    args = json.loads(tool_call.arguments)
    
    # drop filling string statement of cal_id
    if tool_call.name == 'get_upcoming_event':
        print(args['cal_id'])
        id_has_num = any(char.isdigit() for char in args['cal_id'])
        if not id_has_num: args.pop('cal_id', None)

    # call function + get output
    result = call_function(name, args)
    input_message.append(tool_call)
    input_message.append({
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result)
    })

Step 5: Call the same client again, but now with the function outputs tacked on to the input message.   
In this scenario below, we are:
1. asking the question
2. the model will request to use one of the tools
3. we respond you already used it and this was the provided output.... so you have the info you need to answer the question and don't actually have to make a tool call.

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_message,
    tools=(calendar_get_events + weather_tool),
)

In [ ]:
input_msg = "Please schedule a block of time or a run based on your response of the best day and time to run: " + response.output_text
print("Here is our new input message:", input_msg)

### Call Agent #3 to Schedule Event

Here we are including the input message from above that contains all the information needed to schedule the event. Now we just have the model schedule it based on the information provided.

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_msg,
    tools=(schedule_event),
)

In [ ]:
import calendars
for tool_call in response.output:
    # change arguments from string into json
    args = json.loads(tool_call.arguments)
    calendars.write_event(**args)
    print('Calendar updated!')